### **Implementation of tree model**

**Preprocessing:**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import tree
import random

## Loading datasets
train = pd.read_csv('titanic-train.csv')
test = pd.read_csv('titanic-test.csv')

train.info()
train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Embarked  889 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Fare      891 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


,Survived,Pclass,Embarked,Sex,Age,SibSp,Parch,Fare
0,0,3,S,male,22.0,1,0,7.2500
1,1,1,C,female,38.0,1,0,71.2833
2,1,3,S,female,26.0,0,0,7.9250
3,1,1,S,female,35.0,1,0,53.1000
4,0,3,S,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
886,0,2,S,male,27.0,0,0,13.0000
887,1,1,S,female,19.0,0,0,30.0000
888,0,3,S,female,NaN,1,2,23.4500
889,1,1,C,male,26.0,0,0,30.0000


In [4]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
# encoder.fit_transform(array)

def prepro(data, isTest):
  try:
      # Remove useless columns
      data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
      if isTest:
          data = data.drop(['Survived'], axis = 1)
      # Filling empty numerical values with median
      data['Age'] = data['Age'].fillna(data['Age'].median())
      data['Fare'] = data['Fare'].fillna(data['Fare'].median())
      # Filling empty embarked with S
      data['Embarked'] = data['Embarked'].fillna('S')
      # Replacing categorical sex with integer values (0 for F and 1 for M)
      data['Sex'] = encoder.fit_transform(data['Sex'])
      # Replacing categorical Embarked with integer values (0 for C, 1 for Q and 2 for S)
      data['Embarked'] = encoder.fit_transform(data['Embarked'])
  except KeyError as e:
      print('Invalid preprocess')
      print(e)

      if isTest:
          data = data.drop(['Survived'], axis = 1)
      # Filling empty numerical values with median
      data['Age'] = data['Age'].fillna(data['Age'].median())
      data['Fare'] = data['Fare'].fillna(data['Fare'].median())
      # Filling empty embarked with S
      data['Embarked'] = data['Embarked'].fillna('S')
      # Replacing categorical sex with integer values (0 for F and 1 for M)
      data['Sex'] = encoder.fit_transform(data['Sex'])
      # Replacing categorical Embarked with integer values (0 for C, 1 for Q and 2 for S)
      data['Embarked'] = encoder.fit_transform(data['Embarked'])

      # make the ages discrete
      age = data['Age']
      for i in range(len(age)):
        if age[i] < 15:   # child -> 0
          age[i] = 0
        elif (age[i] > 15 or age[i] == 15) and age[i] < 30:    # teenager -> 1
          age[i] = 1
        elif (age[i] > 30 or age[i] == 30) and age[i] < 55:    # adult -> 2
          age[i] = 2
        elif age[i] > 55 or age[i] == 55:    # old -> 3
          age[i] = 3

      data['Age'] = age

      # make the fare discrete
      fare = data['Fare']
      for i in range(len(fare)):
        if fare[i] < 50:   # cheap -> 0
          fare[i] = 0
        elif (fare[i] > 50 or fare[i] == 50) and fare[i] < 100:    # suitable -> 1
          fare[i] = 1
        elif (fare[i] > 100 or fare[i] == 100):    # expensive -> 2
          fare[i] = 2

      data['Fare'] = fare
  return data

In [5]:
train = prepro(train, False)
train.head()

Invalid preprocess
"['PassengerId' 'Name' 'Ticket' 'Cabin'] not found in axis"


<ipython-input-4-1d5ae61d1680>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age[i] = 1
<ipython-input-4-1d5ae61d1680>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age[i] = 2
<ipython-input-4-1d5ae61d1680>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age[i] = 0
<ipython-input-4-1d5ae61d1680>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

,Survived,Pclass,Embarked,Sex,Age,SibSp,Parch,Fare
0,0,3,2,1,1.0,1,0,0.0
1,1,1,0,0,2.0,1,0,1.0
2,1,3,2,0,1.0,0,0,0.0
3,1,1,2,0,2.0,1,0,1.0
4,0,3,2,1,2.0,0,0,0.0


In [6]:
test = prepro(test, False)
test.head()

Invalid preprocess
"['PassengerId' 'Name' 'Ticket'] not found in axis"


<ipython-input-4-1d5ae61d1680>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age[i] = 2
<ipython-input-4-1d5ae61d1680>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age[i] = 3
<ipython-input-4-1d5ae61d1680>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age[i] = 1
<ipython-input-4-1d5ae61d1680>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,1,2.0,0,0,0.0,NaN,1
1,3,0,2.0,1,0,0.0,NaN,2
2,2,1,3.0,0,0,0.0,NaN,1
3,3,1,1.0,0,0,0.0,NaN,2
4,3,0,1.0,1,1,0.0,NaN,2


**Number of Features = 3**

In [7]:
x = train.drop(['Survived'], axis=1).values   ## train data without labels
y = train['Survived'].values                  ## labels of train data

num_features = 3

features_list = list(train)
print(features_list)
features_indx = random.sample(range(1,len(features_list)), num_features)
print(features_indx)
features_list = np.array(features_list)
features = features_list[np.array(features_indx)]
print("selected features:")
print(features)

['Survived', 'Pclass', 'Embarked', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
[2, 1, 5]
selected features:
['Embarked' 'Pclass' 'SibSp']


In [8]:
rows = [i for i in range(0, train.shape[0])]
columns = features_indx
train_dataset = np.array(train)
root = DecisionTree(train_dataset, rows, columns, features_list)
root.decision = 'Start'
Leaves(root)

Start
Pclass
3.0
Embarked
2.0
SibSp
1.0
died
0.0
died
3.0
died
2.0
died
4.0
died
5.0
died
8.0
died
1.0
SibSp
0.0
died
4.0
died
1.0
died
2.0
survived
0.0
SibSp
0.0
died
1.0
died
2.0
died
1.0
SibSp
1.0
Embarked
0.0
died
2.0
died
1.0
survived
0.0
Embarked
2.0
died
0.0
died
3.0
Embarked
2.0
died
2.0
Embarked
1.0
died
0.0
survived
2.0
survived
2.0
SibSp
1.0
Embarked
0.0
died
2.0
died
0.0
Embarked
2.0
died
0.0
died
1.0
died
2.0
Embarked
2.0
died
3.0
survived


**Number of Features = 5**

In [9]:
x = train.drop(['Survived'], axis=1).values   ## train data without labels
y = train['Survived'].values                  ## labels of train data

num_features = 5

features_list = list(train)
print(features_list)
features_indx = random.sample(range(1,len(features_list)), num_features)
print(features_indx)
features_list = np.array(features_list)
features = features_list[np.array(features_indx)]
print("selected features:")
print(features)

['Survived', 'Pclass', 'Embarked', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
[4, 7, 6, 3, 5]
selected features:
['Age' 'Fare' 'Parch' 'Sex' 'SibSp']


In [10]:
rows = [i for i in range(0, train.shape[0])]
columns = features_indx
train_dataset = np.array(train)
root = DecisionTree(train_dataset, rows, columns, features_list)
root.decision = 'Start'
Leaves(root)

Start
Sex
1.0
Age
1.0
Fare
0.0
Parch
0.0
SibSp
1.0
died
0.0
died
2.0
died
1.0
SibSp
1.0
died
3.0
died
4.0
died
2.0
died
0.0
died
3.0
died
2.0
died
2.0
Parch
2.0
SibSp
3.0
died
0.0
died
1.0
died
0.0
died
1.0
SibSp
1.0
Parch
0.0
died
0.0
Parch
0.0
died
1.0
died
2.0
died
8.0
died
2.0
Fare
0.0
Parch
0.0
SibSp
0.0
died
1.0
died
2.0
died
5.0
died
2.0
died
1.0
died
4.0
died
1.0
SibSp
0.0
Parch
0.0
died
1.0
died
1.0
Parch
0.0
died
1.0
died
2.0
died
2.0
Parch
1.0
SibSp
0.0
died
1.0
died
2.0
survived
0.0
SibSp
1.0
died
2.0
survived
0.0
survived
0.0
SibSp
3.0
died
4.0
Parch
1.0
died
2.0
Fare
0.0
died
5.0
died
0.0
Parch
2.0
survived
0.0
died
1.0
survived
1.0
survived
2.0
survived
3.0
Fare
0.0
Parch
0.0
SibSp
0.0
died
1.0
died
1.0
SibSp
0.0
died
1.0
Parch
1.0
died
2.0
died
0.0
Fare
1.0
SibSp
1.0
survived
0.0
survived
8.0
died
2.0
survived
0.0
SibSp
0.0
Parch
0.0
Age
1.0
died
3.0
died
0.0
died
2.0
died
2.0
Age
1.0
died
0.0
died
2.0
died
1.0
Age
2.0
died
0.0
died
1.0
died
4.0
died
5.0
died
3.0
surviv

**Number of Features = 7**

In [11]:
x = train.drop(['Survived'], axis=1).values   ## train data without labels
y = train['Survived'].values                  ## labels of train data

num_features = 7

features_list = list(train)
print(features_list)
features_indx = random.sample(range(1,len(features_list)), num_features)
print(features_indx)
features_list = np.array(features_list)
features = features_list[np.array(features_indx)]
print("selected features:")
print(features)

['Survived', 'Pclass', 'Embarked', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
[1, 3, 5, 2, 4, 7, 6]
selected features:
['Pclass' 'Sex' 'SibSp' 'Embarked' 'Age' 'Fare' 'Parch']


In [12]:
rows = [i for i in range(0, train.shape[0])]
columns = features_indx
train_dataset = np.array(train)
root = DecisionTree(train_dataset, rows, columns, features_list)
root.decision = 'Start'
Leaves(root)

Start
Sex
1.0
Age
1.0
Pclass
3.0
Embarked
2.0
Fare
0.0
Parch
0.0
SibSp
1.0
died
0.0
died
2.0
died
3.0
died
2.0
died
1.0
died
1.0
SibSp
8.0
died
0.0
Parch
0.0
died
1.0
SibSp
0.0
Fare
0.0
Parch
0.0
died
1.0
died
2.0
survived
0.0
Parch
0.0
SibSp
0.0
Fare
0.0
died
2.0
died
1.0
died
1.0
SibSp
1.0
Fare
0.0
died
2.0
SibSp
0.0
Embarked
2.0
Fare
0.0
Parch
0.0
died
1.0
died
1.0
died
0.0
Fare
0.0
Parch
0.0
died
1.0
died
2.0
died
1.0
Fare
0.0
Embarked
2.0
SibSp
0.0
Parch
0.0
died
0.0
SibSp
0.0
Parch
0.0
died
2.0
Parch
2.0
SibSp
3.0
died
0.0
Embarked
0.0
died
1.0
died
0.0
died
1.0
Embarked
0.0
Parch
0.0
SibSp
1.0
died
0.0
died
1.0
survived
2.0
SibSp
0.0
died
1.0
Parch
0.0
died
2.0
Pclass
3.0
Fare
0.0
Embarked
2.0
SibSp
0.0
Parch
0.0
died
2.0
died
1.0
died
1.0
died
2.0
died
0.0
died
1.0
died
1.0
survived
1.0
Parch
0.0
Embarked
2.0
Fare
1.0
SibSp
0.0
died
1.0
died
0.0
SibSp
0.0
died
2.0
survived
0.0
SibSp
0.0
Fare
0.0
died
2.0
survived
1.0
died
1.0
Fare
1.0
survived
2.0
died
1.0
died
1.0
Fare
1.0
Sib

### **Decision Tree (ID3 method):**

In [13]:
import math
import copy

## A function for entropy calculation:
def Entropy(data, rows):
    survived = 0
    died = 0
    ans = -1
    entropy = 0
    for r in rows:
        if data[r][0] == 1:
            survived = survived + 1
        else:
            died = died + 1

    x = survived/(survived+died)
    y = died/(survived+died)
    if x != 0 and y != 0:
        entropy = -1 * (x*math.log2(x) + y*math.log2(y))
    if x == 1:
        ans = 1
    if y == 1:
        ans = 0
    return [entropy, ans]

# ------------------------------------------------------------------------------
## A function for finding maximum Information Gain:
def MaxIG(data, rows, columns):
    maxGain = 0
    max_indx = 0
    [entropy, ans] = Entropy(data, rows)
    if entropy == 0:
        return maxGain, max_indx, ans

    for c in columns:
        mydict = {}
        col = c
        for r in rows:
            key = data[r][col]
            if key not in mydict:
                mydict[key] = 1
            else:
                mydict[key] = mydict[key] + 1

        gain = entropy

        for key in mydict:
            survived = 0
            died = 0
            for k in rows:
                if data[k][c] == key:
                    if data[k][0] == 1:
                        survived = survived + 1
                    else:
                        died = died + 1

            x = survived/(survived+died)
            y = died/(survived+died)
            if x != 0 and y != 0:
                gain += (mydict[key]/data.shape[0] * (x*math.log2(x) + y*math.log2(y)))
        if gain > maxGain:
            maxGain = gain
            max_indx = c

    return maxGain, max_indx, ans

# ------------------------------------------------------------------------------
## Creating the nodes of decision tree:
class Node(object):
    def __init__(self):
        self.value = None
        self.decision = None
        self.leaves = None

# ------------------------------------------------------------------------------
## A function for creating a decision tree:
def DecisionTree(data, rows, columns, features):

    maxGain, max_indx, ans = MaxIG(data, rows, columns)
    root = Node()
    root.leaves = []

    if maxGain == 0:
        if ans == 1:
            root.value = 'survived'  # for survived ones
        else:
            root.value = 'died'  # for died ones
        return root

    root.value = features[max_indx]
    mydict = {}
    for r in rows:
        key = data[r][max_indx]
        if key not in mydict:
            mydict[key] = 1
        else:
            mydict[key] += 1
    
    newcolumns = copy.deepcopy(columns)
    newcolumns.remove(max_indx)
    for key in mydict:
        newrows = []
        for i in rows:
            if data[i][max_indx] == key:
                newrows.append(i)
        temp = DecisionTree(data, newrows, newcolumns, features)
        temp.decision = key
        root.leaves.append(temp)
    return root

# ------------------------------------------------------------------------------
## A function for creating the leaves:
def Leaves(root):
    print(root.decision)
    print(root.value)

    n = len(root.leaves)
    if n > 0:
        for i in range(0, n):
            Leaves(root.leaves[i])